In [2]:
# we must import the libraries once again since we haven't imported them in this file
import numpy as np
import tensorflow as tf

In [3]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype('float')
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype('int')


# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype('float'), npz['targets'].astype('int')

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype('float'), npz['targets'].astype('int')

In [7]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100
# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)


# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
           callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Epoch 1/100
36/36 - 1s - loss: 0.5970 - accuracy: 0.6703 - val_loss: 0.5123 - val_accuracy: 0.7450 - 1s/epoch - 40ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4749 - accuracy: 0.7552 - val_loss: 0.4381 - val_accuracy: 0.7696 - 145ms/epoch - 4ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4194 - accuracy: 0.7818 - val_loss: 0.4125 - val_accuracy: 0.7517 - 132ms/epoch - 4ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3919 - accuracy: 0.7863 - val_loss: 0.3866 - val_accuracy: 0.7718 - 125ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3753 - accuracy: 0.7946 - val_loss: 0.3837 - val_accuracy: 0.7696 - 127ms/epoch - 4ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3682 - accuracy: 0.7999 - val_loss: 0.3734 - val_accuracy: 0.7897 - 139ms/epoch - 4ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3606 - accuracy: 0.8013 - val_loss: 0.3683 - val_accuracy: 0.7740 - 167ms/epoch - 5ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3519 - accuracy: 0.8173 - val_loss: 0.3651 - val_accuracy: 0.8009 - 137ms/epoch - 4ms/step
Epoch 9/10

In [8]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 4ms/step - loss: 0.3473 - accuracy: 0.8080


In [9]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.35. Test accuracy: 80.80%
